In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/Users/andrew/Documents/python/pygplates_rev28_python38_MacOS64')
import pygplates

from collections import defaultdict

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.cm as cm
import matplotlib as mpl


In [2]:
datadir = '/Users/andrew/Documents/Data/plate_boundary_analysis/'
platedir = '/Users/andrew/Documents/GitHub/EarthBytePlateMotionModel-ARCHIVE/Muller++_2015_AREPS_CORRECTED/'
savedir = '/Users/andrew/Documents/Data/Gernon_shallow_rdiges/'

# import previously made file in 'extract velocities'
file = '%sPlateBoundaryTypes_400-0_25Oct19.h5' % datadir
df = pd.read_hdf(file,'Statistics_table')
tags = df['FeatureType']

rotfile = '%sGlobal_EarthByte_230-0Ma_GK07_AREPS.rot' % platedir
COBs = '%sAgeGridInput/Global_EarthByte_230-0Ma_GK07_AREPS_COB_Terranes.gpml' % platedir
coastlines = '%sGlobal_EarthByte_230-0Ma_GK07_AREPS_Coastlines.gpml' % platedir
input_topology_filename = ['%sGlobal_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpml' % platedir,
                           '%sGlobal_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpml' % platedir]

rotation_model = pygplates.RotationModel(rotfile)


In [7]:
#top | left | bottom | right | start time | left plate | right plate
# start time | plate IDs
#ages from Seton et al. 2012
MOR_database = []

#Labrador_Sea = [60, 101, 102]
#North_Atlantic = [56, 302, 102, 318, 301, 907] 
#Central_Atlantic = [180, 101, 714, 701]
Equatorial_Atlantic = [120, 201, 714]#but difficult
South_Atlantic = [132, 201, 701]
#Australia_Molucca = [155, 780, 781, 678]
#MadInd_Africa = [154, 501, 701]
#Ind_Australia = [129, 802, 501, 801, 550]
#Madagascar_India = [87, 702, 701, 501]
#Ant_Africa = [154, 701, 802, 28900, 201]
#Aus_Antarctica = [40, 802, 801]

In [8]:
#turn into dictionary for ease
ridge_dict = {
              'Ind_Australia': Ind_Australia,
              'Aus_Antarctica': Aus_Antarctica}

dict_names = [key for key in ridge_dict ]

#turn into dictionary for ease
#ridge_dict = {'Labrador_Sea': Labrador_Sea,
#              'North_Atlantic': North_Atlantic,
#              'Central_Atlantic': Central_Atlantic,
#              'Equatorial_Atlantic': Equatorial_Atlantic,
#              'South_Atlantic': South_Atlantic,
#              'Australia_Molucca': Australia_Molucca,
#              'MadInd_Africa': MadInd_Africa,
#              'Ind_Australia': Ind_Australia,
#              'Madagascar_India': Madagascar_India,
#              'Ant_Africa': Ant_Africa,
#              'Aus_Antarctica': Aus_Antarctica}
#
#dict_names = [key for key in ridge_dict ]

In [9]:
dict_names

['Ind_Australia', 'Aus_Antarctica']

In [13]:
SR = 'Spreading Rate'
length = 'Boundary-Length'
StartLat = 'Start-Lat-Point'
StartLon = 'Start-Lon-Point'
EndLat = 'End-Lat-Point'
EndLon = 'End-Lon-Point'
MeanLat = 'Mean-Lat-Point'
MeanLon = 'Mean-Lon-Point'
RightPlateID = 'RightPlate'
LeftPlateID = 'LeftPlate'
Index = 'Index'

# deviation angle (in degrees) to split transforms vs. ridges from
deviation_angle= 70
#time resolution to extract data
time_resolution = 1
time_duration = 30
indices = []
for ind,(ridge,j) in enumerate(ridge_dict.items()):
    print(ind, ridge,j)
    MOR_key = ridge
    start_time = j[0]
    plate_IDs = j[1:]
    plate_IDs = [str(i) for i in plate_IDs]
   # print(ridge)
    #construct valid times
    times = np.arange(start_time, start_time-time_duration-time_resolution, -1)
    print(times)
    seafloor_production[ind] = defaultdict(lambda: defaultdict(list))

    #print(seafloor_production)
    for time in times:
    #print (time,  'Ma')
    #we only need to cut up spreading ridges by velocity
        subset = df[(df['Time_Ma']>=time)
                  & (df['Time_Ma']<(time+time_resolution))
                  & (df['FeatureType']=='gpml:MidOceanRidge')
                  & (np.abs(df['Deviation_mod_deg'])<=deviation_angle)
                  & (df['LeftPlate'].isin(plate_IDs))
                  & (df['RightPlate'].isin(plate_IDs))
                    ]
        seafloor_production[ind][time][SR] = []
        seafloor_production[ind][time][length] = []
        seafloor_production[ind][time][StartLat] = []
        seafloor_production[ind][time][StartLon] = []
        seafloor_production[ind][time][EndLat] = []
        seafloor_production[ind][time][EndLon] = []
        seafloor_production[ind][time][MeanLat] = []
        seafloor_production[ind][time][MeanLon] = []
        seafloor_production[ind][time][RightPlateID] = []
        seafloor_production[ind][time][LeftPlateID] = []
        seafloor_production[ind][time][Index] = []

        for index, row in subset.iterrows():
            #print(row.EndPointLat)
            seafloor_production[ind][time][SR].append(row.Plate_Velocity)
            seafloor_production[ind][time][length].append(row.Length_km)
            seafloor_production[ind][time][StartLat].append(row.StartPointLat)
            seafloor_production[ind][time][StartLon].append(row.StartPointLon)
            seafloor_production[ind][time][EndLat].append(row.EndPointLat)
            seafloor_production[ind][time][EndLon].append(row.EndPointLon)
            seafloor_production[ind][time][MeanLat].append(np.mean([row.EndPointLat, row.StartPointLat]))
            seafloor_production[ind][time][MeanLon].append(np.mean([row.EndPointLon, row.StartPointLon]))
            seafloor_production[ind][time][RightPlateID].append(row.RightPlate)
            seafloor_production[ind][time][LeftPlateID].append(row.LeftPlate)
            seafloor_production[ind][time][Index].append(row.index)
        
    indices.append(ind)
    #ind += 1
    

0 Ind_Australia [129, 802, 501, 801, 550]
[129 128 127 126 125 124 123 122 121 120 119 118 117 116 115 114 113 112
 111 110 109 108 107 106 105 104 103 102 101 100  99]
1 Aus_Antarctica [40, 802, 801]
[40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17
 16 15 14 13 12 11 10]


In [ ]:
segment_list = []
summed_production = []
plotting = []
for ind in indices:
    times = list(seafloor_production[ind].keys())
    for time in times:
        seafloor_production[ind][time]
        tmp_production = []
        for ind2,value in enumerate(seafloor_production[ind][time]['Spreading Rate']):
            segment_list.append((dict_names[ind],
                                 time,
                                 seafloor_production[ind][time]['Spreading Rate'][ind2],
                                 seafloor_production[ind][time]['Boundary-Length'][ind2],
                                 seafloor_production[ind][time]['Start-Lat-Point'][ind2],
                                 seafloor_production[ind][time]['Start-Lon-Point'][ind2],
                                 seafloor_production[ind][time]['End-Lat-Point'][ind2],
                                 seafloor_production[ind][time]['End-Lon-Point'][ind2],
                                 seafloor_production[ind][time]['RightPlate'][ind2],
                                 seafloor_production[ind][time]['LeftPlate'][ind2]))
            
            plotting.append((dict_names[ind],
                            time,
                            seafloor_production[ind][time]['Spreading Rate'][ind2], 
                            seafloor_production[ind][time]['Boundary-Length'][ind2],
                            (seafloor_production[ind][time]['Start-Lat-Point'][ind2],
                                         seafloor_production[ind][time]['Start-Lon-Point'][ind2]),
                            (seafloor_production[ind][time]['End-Lat-Point'][ind2],
                                         seafloor_production[ind][time]['End-Lon-Point'][ind2]),
                             seafloor_production[ind][time]['RightPlate'][ind2],
                             seafloor_production[ind][time]['LeftPlate'][ind2]))
            
            tmp_production.append(seafloor_production[ind][time]['Spreading Rate'][ind2] * 10 * \
                                  seafloor_production[ind][time]['Boundary-Length'][ind2])
            
        summed_production.append((dict_names[ind], time, np.sum(tmp_production)))

In [ ]:
full_results_columns = ('RidgeSystem', 'Time_Ma','SpreadingRate_cm/a','Length_km',
                         'StartLat','StartLon', 'EndLat','EndLon', 'RightPlate','LeftPlate')

summed_results_columns = ('RidgeSystem', 'Time_Ma','production km2/Ma')

plotting_columns = ('RidgeSystem', 'Time_Ma','SpreadingRate_cm/a','Length_km',
                         'StartPoint','EndPoint', 'RightPlate', 'LeftPlate')

df_full_results = pd.DataFrame(segment_list, columns=full_results_columns)
df_summed_results = pd.DataFrame(summed_production, columns=summed_results_columns)
df_plotting = pd.DataFrame(plotting, columns=plotting_columns)

In [ ]:
df_full_results.to_csv('full_results_Cretaceous.csv')
df_summed_results.to_csv('summed_results_Cretaceous.csv')

In [ ]:
wrapped_lon_left

In [ ]:
plt.plot(wrapped_lon_left,
                 wrapped_lat_left,
                c=colors[from_time])

In [ ]:
data_crs = ccrs.Geodetic()
poly_data_crs = ccrs.PlateCarree()
date_line_wrapper = pygplates.DateLineWrapper(180) 

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(24,12),subplot_kw={'projection': ccrs.Robinson(180)})

reconstructed_coastlines = []
resolved_topologies = []
shared_boundary_sections = []
reconstructed_COBs = []


pygplates.reconstruct(COBs,rotation_model, reconstructed_COBs, 0)
pygplates.reconstruct(coastlines,rotation_model,reconstructed_coastlines,0)
pygplates.resolve_topologies(input_topology_filename, rotation_model,
                             resolved_topologies, 0, shared_boundary_sections)

colors = cm.viridis(np.linspace(0, 1, 181))
timestep = 1 
for ind,(i,j) in enumerate(zip(df_plotting['StartPoint'], df_plotting['EndPoint'])):
    
    right_plate = int(df_plotting['RightPlate'][ind])
    left_plate = int(df_plotting['LeftPlate'][ind])
    from_time = int(df_plotting['Time_Ma'][ind])

    
    points = np.array([i[0], i[1], j[0], j[1]])
    polyline = pygplates.PolylineOnSphere(zip(points[::2],points[1::2]))


    rotation_right = rotation_model.get_rotation(0, right_plate, from_time)
    rotation_left = rotation_model.get_rotation(0, left_plate, from_time)
    reconstructed_line_right = rotation_right * polyline
    reconstructed_line_left = rotation_left * polyline
    
    if from_time % 5 == 0:
        
        wrapped_line_left = date_line_wrapper.wrap(reconstructed_line_left, 1)
        for wrapped_line in wrapped_line_left:
            wrapped_lat_left = []
            wrapped_lon_left = []
            for point in wrapped_line.get_points():
                wrapped_lat_left.append(point.get_latitude())
                wrapped_lon_left.append(point.get_longitude())
            
            ax.scatter(wrapped_lon_left, wrapped_lat_left,
                    color=colors[from_time],
                    transform=poly_data_crs)
                
        wrapped_line_right = date_line_wrapper.wrap(reconstructed_line_right, 1)
        for wrapped_line in wrapped_line_right:
            wrapped_lat_right = []
            wrapped_lon_right = []
            for point in wrapped_line.get_points():
                wrapped_lat_right.append(point.get_latitude())
                wrapped_lon_right.append(point.get_longitude())
            


            ax.scatter(wrapped_lon_right,
                 wrapped_lat_right,
                color=colors[from_time],
                transform=poly_data_crs)

In [ ]:
Ind_Australia

In [ ]:
data_crs = ccrs.PlateCarree()
poly_data_crs = ccrs.PlateCarree()
date_line_wrapper = pygplates.DateLineWrapper(180) 

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(24,12),subplot_kw={'projection': ccrs.Robinson(180)})

reconstructed_coastlines = []
resolved_topologies = []
shared_boundary_sections = []
reconstructed_COBs = []


pygplates.reconstruct(COBs,rotation_model, reconstructed_COBs, 99)
pygplates.reconstruct(coastlines,rotation_model,reconstructed_coastlines,99)
pygplates.resolve_topologies(input_topology_filename, rotation_model,
                             resolved_topologies, 99, shared_boundary_sections)

colors = cm.viridis(np.linspace(0, 1, 181))
timestep = 1 
for ind,(i,j) in enumerate(zip(df_plotting['StartPoint'], df_plotting['EndPoint'])):
    
    right_plate = int(df_plotting['RightPlate'][ind])
    left_plate = int(df_plotting['LeftPlate'][ind])
    from_time = int(df_plotting['Time_Ma'][ind])

    
    points = np.array([i[0], i[1], j[0], j[1]])
    polyline = pygplates.PolylineOnSphere(zip(points[::2],points[1::2]))


    rotation_right = rotation_model.get_rotation(99, right_plate, from_time)
    rotation_left = rotation_model.get_rotation(99, left_plate, from_time)
    reconstructed_line_right = rotation_right * polyline
    reconstructed_line_left = rotation_left * polyline
    
    if from_time % 5 == 0:
        
        wrapped_line_left = date_line_wrapper.wrap(reconstructed_line_left, 1)
        for wrapped_line in wrapped_line_left:
            wrapped_lat_left = []
            wrapped_lon_left = []
            for point in wrapped_line.get_points():
                wrapped_lat_left.append(point.get_latitude())
                wrapped_lon_left.append(point.get_longitude())
            
            ax.scatter(wrapped_lon_left,
                 wrapped_lat_left,
                color=colors[from_time],
                transform=poly_data_crs)
                
        wrapped_line_right = date_line_wrapper.wrap(reconstructed_line_right, 1)
        for wrapped_line in wrapped_line_right:
            wrapped_lat_right = []
            wrapped_lon_right = []
            for point in wrapped_line.get_points():
                wrapped_lat_right.append(point.get_latitude())
                wrapped_lon_right.append(point.get_longitude())
            


            ax.scatter(wrapped_lon_right,
                 wrapped_lat_right,
                color=colors[from_time],
                transform=poly_data_crs)
        
for coastline in reconstructed_coastlines:
    coastline_geom = pygplates.PolygonOnSphere(coastline.get_reconstructed_geometry())
    wrapped_coastlines = date_line_wrapper.wrap(coastline_geom, 1)
    for wrapped_coastline in wrapped_coastlines:
        wrapped_coastlines_lat = []
        wrapped_coastlines_lon = []
        for wrapped_coastline_point in wrapped_coastline.get_exterior_points():
            wrapped_coastline_lat_lon = wrapped_coastline_point.get_latitude(), wrapped_coastline_point.get_longitude()
            wrapped_coastlines_lat.append(wrapped_coastline_point.get_latitude())
            wrapped_coastlines_lon.append(wrapped_coastline_point.get_longitude())
        ax.scatter(wrapped_coastlines_lon, wrapped_coastlines_lat, color='k', lw=0.1, transform=data_crs)

for polygon in reconstructed_COBs:
    poly_geom = pygplates.PolygonOnSphere(polygon.get_reconstructed_geometry())
    wrapped_polys = date_line_wrapper.wrap(poly_geom, 1)

    for wrapped_poly in wrapped_polys:
        wrapped_poly_lat = []
        wrapped_poly_lon = []
        for wrapped_poly_point in wrapped_poly.get_exterior_points():
            wrapped_poly_lat.append(wrapped_poly_point.get_latitude())
            wrapped_poly_lon.append(wrapped_poly_point.get_longitude())
        ax.scatter(wrapped_poly_lon, wrapped_poly_lat, color='k', linewidth=1 ,zorder=0, transform=poly_data_crs)
        #ax.fill(wrapped_poly_lon, wrapped_poly_lat, color='grey', alpha=1, linewidth=1 ,zorder=0, transform=data_crs)

for shared_boundary_section in shared_boundary_sections:
    if shared_boundary_section.get_feature().get_feature_type() == pygplates.FeatureType.gpml_mid_ocean_ridge:
        for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
            tmp = shared_sub_segment.get_resolved_geometry()
            wrapped_polylines = date_line_wrapper.wrap(tmp)
            for wrapped_polyline in wrapped_polylines:
                mor_wrapped_point_lat = []
                mor_wrapped_point_lon = []
                for wrapped_point in wrapped_polyline.get_points():
                    mor_wrapped_point_lat.append(wrapped_point.get_latitude())
                    mor_wrapped_point_lon.append(wrapped_point.get_longitude())
                ax.scatter(mor_wrapped_point_lon,mor_wrapped_point_lat, color='#545eb3',
                        linewidth=5, alpha=1, zorder=2, transform=data_crs)
                
    elif shared_boundary_section.get_feature().get_feature_type() == pygplates.FeatureType.gpml_subduction_zone:
        for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
            tmp = shared_sub_segment.get_resolved_geometry()
            wrapped_polylines = date_line_wrapper.wrap(tmp)
            for wrapped_polyline in wrapped_polylines:
                mor_wrapped_point_lat = []
                mor_wrapped_point_lon = []
                for wrapped_point in wrapped_polyline.get_points():
                    mor_wrapped_point_lat.append(wrapped_point.get_latitude())
                    mor_wrapped_point_lon.append(wrapped_point.get_longitude())
                ax.scatter(mor_wrapped_point_lon,mor_wrapped_point_lat, color='#853a2b',
                        linewidth=5, alpha=1, zorder=2, transform=data_crs)                
    else:
        for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
            tmp = shared_sub_segment.get_resolved_geometry()
            wrapped_polylines = date_line_wrapper.wrap(tmp)
            for wrapped_polyline in wrapped_polylines:
                mor_wrapped_point_lat = []
                mor_wrapped_point_lon = []
                for wrapped_point in wrapped_polyline.get_points():
                    mor_wrapped_point_lat.append(wrapped_point.get_latitude())
                    mor_wrapped_point_lon.append(wrapped_point.get_longitude())
                ax.scatter(mor_wrapped_point_lon,mor_wrapped_point_lat, color='k',
                        linewidth=2, alpha=1, zorder=2, transform=data_crs)
        
ax.set_global()
norm = mpl.colors.Normalize(vmin=0,vmax=180)
cmap = plt.get_cmap('viridis',180)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, boundaries=np.arange(0,200,20))

#plt.savefig('present_day_global.pdf')

In [ ]:
ridge_dict

In [ ]:
atlantic_plotting = df_plotting.loc[(df_plotting['RidgeSystem'] == 'North_Atlantic') |
                                   (df_plotting['RidgeSystem'] == 'Labrador_Sea') |
                                   (df_plotting['RidgeSystem'] == 'Central_Atlantic') |
                                   (df_plotting['RidgeSystem'] == 'Equatorial_Atlantic') |
                                   (df_plotting['RidgeSystem'] == 'South_Atlantic')].reset_index(drop=True)

In [ ]:
Antarctica_plotting = df_plotting.loc[(df_plotting['RidgeSystem'] == 'Ant_Africa') |
                                      (df_plotting['RidgeSystem'] == 'Aus_Antarctica')].reset_index(drop=True)

In [ ]:
ridge_dict

In [ ]:
coastline.get_reconstructed_geometry().to_lat_lon_array()[::,1]

In [ ]:
data_crs = ccrs.Geodetic()
poly_data_crs = ccrs.PlateCarree()

date_line_wrapper = pygplates.DateLineWrapper(-30) 

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(24,12),
                       subplot_kw={'projection': ccrs.Orthographic(central_longitude=-30, central_latitude=-90)})


ridge_system = Aus_Antarctica

reconstructed_coastlines = []
resolved_topologies = []
shared_boundary_sections = []
reconstructed_COBs = []


pygplates.reconstruct(COBs,rotation_model, reconstructed_COBs, ridge_system[0]-30)
pygplates.reconstruct(coastlines,rotation_model,reconstructed_coastlines,ridge_system[0]-30)
pygplates.resolve_topologies(input_topology_filename, rotation_model,
                             resolved_topologies, ridge_system[0]-30, shared_boundary_sections)

color = cm.viridis(np.linspace(0, 1, 181))
timestep = 1 

#plotting = df_plotting.loc[(df_plotting['RidgeSystem'] == 'North_Atlantic') |
#                           (df_plotting['RidgeSystem'] == 'Labrador_Sea') |
#                           (df_plotting['RidgeSystem'] == 'Central_Atlantic') |
#                           (df_plotting['RidgeSystem'] == 'Equatorial_Atlantic') |
#                           (df_plotting['RidgeSystem'] == 'South_Atlantic')].reset_index(drop=True)

plotting = df_plotting.loc[(df_plotting['RidgeSystem'] == 'Ant_Africa') |
                           (df_plotting['RidgeSystem'] == 'Aus_Antarctica')].reset_index(drop=True)

for ind,(i,j) in enumerate(zip(plotting['StartPoint'], plotting['EndPoint'])):
    #print( north_atlantic_plotting['RightPlate'][ind])
    right_plate = int(plotting['RightPlate'][ind])
    left_plate = int(plotting['LeftPlate'][ind])
    from_time = int(plotting['Time_Ma'][ind])

    
    points = np.array([i[0], i[1], j[0], j[1]])
    polyline = pygplates.PolylineOnSphere(zip(points[::2],points[1::2]))


    rotation_right = rotation_model.get_rotation(ridge_system[0]-30, right_plate, from_time)
    rotation_left = rotation_model.get_rotation(ridge_system[0]-30, left_plate, from_time)
    reconstructed_line_right = rotation_right * polyline
    reconstructed_line_left = rotation_left * polyline
    
    if from_time % 5 == 0:
        
        wrapped_line_left = date_line_wrapper.wrap(reconstructed_line_left, 0.1)
        for wrapped_line in wrapped_line_left:
            wrapped_lat_left = []
            wrapped_lon_left = []
            for point in wrapped_line.get_points():
                wrapped_lat_left.append(point.get_latitude())
                wrapped_lon_left.append(point.get_longitude())
            
            ax.plot(wrapped_lon_left,
                 wrapped_lat_left,
                c=colors[from_time],
                transform=poly_data_crs)
                
        wrapped_line_right = date_line_wrapper.wrap(reconstructed_line_right, 0.1)
        for wrapped_line in wrapped_line_right:
            wrapped_lat_right = []
            wrapped_lon_right = []
            for point in wrapped_line.get_points():
                wrapped_lat_right.append(point.get_latitude())
                wrapped_lon_right.append(point.get_longitude())
            
            ax.plot(wrapped_lon_right,
                 wrapped_lat_right,
                c=colors[from_time],
                transform=poly_data_crs)
        
for coastline in reconstructed_coastlines:
    poly_lat = coastline.get_reconstructed_geometry().to_lat_lon_array()[::,0]
    poly_lon = coastline.get_reconstructed_geometry().to_lat_lon_array()[::,1]
    ax.plot(poly_lon, poly_lat, color='k', lw=1, transform=data_crs)
#    
for polygon in reconstructed_COBs:
    COB_lat = polygon.get_reconstructed_geometry().to_lat_lon_array()[::,0]
    COB_lon = polygon.get_reconstructed_geometry().to_lat_lon_array()[::,1]
        #ax.plot(wrapped_poly_lon, wrapped_poly_lat, color='k', linewidth=1 ,zorder=0, transform=poly_data_crs)
    ax.plot(COB_lon, COB_lat, color='grey', alpha=1, linewidth=1 ,zorder=0, transform=data_crs)
#
#        
for shared_boundary_section in shared_boundary_sections:
    if shared_boundary_section.get_feature().get_feature_type() == pygplates.FeatureType.gpml_mid_ocean_ridge:
        for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
            tmp = shared_sub_segment.get_resolved_geometry()
            wrapped_polylines = date_line_wrapper.wrap(tmp, 0.1)
            for wrapped_polyline in wrapped_polylines:
                mor_wrapped_point_lat = []
                mor_wrapped_point_lon = []
                for wrapped_point in wrapped_polyline.get_points():
                    mor_wrapped_point_lat.append(wrapped_point.get_latitude())
                    mor_wrapped_point_lon.append(wrapped_point.get_longitude())
                ax.plot(mor_wrapped_point_lon,mor_wrapped_point_lat, color='#545eb3',
                        linewidth=5, alpha=1, zorder=2, transform=data_crs)
                
    elif shared_boundary_section.get_feature().get_feature_type() == pygplates.FeatureType.gpml_subduction_zone:
        for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
            tmp = shared_sub_segment.get_resolved_geometry()
            wrapped_polylines = date_line_wrapper.wrap(tmp, 0.1)
            for wrapped_polyline in wrapped_polylines:
                mor_wrapped_point_lat = []
                mor_wrapped_point_lon = []
                for wrapped_point in wrapped_polyline.get_points():
                    mor_wrapped_point_lat.append(wrapped_point.get_latitude())
                    mor_wrapped_point_lon.append(wrapped_point.get_longitude())
                ax.plot(mor_wrapped_point_lon,mor_wrapped_point_lat, color='#853a2b',
                        linewidth=5, alpha=1, zorder=2, transform=data_crs)                
    else:
        for shared_sub_segment in shared_boundary_section.get_shared_sub_segments():
            tmp = shared_sub_segment.get_resolved_geometry()
            wrapped_polylines = date_line_wrapper.wrap(tmp, 0.1)
            for wrapped_polyline in wrapped_polylines:
                mor_wrapped_point_lat = []
                mor_wrapped_point_lon = []
                for wrapped_point in wrapped_polyline.get_points():
                    mor_wrapped_point_lat.append(wrapped_point.get_latitude())
                    mor_wrapped_point_lon.append(wrapped_point.get_longitude())
                ax.plot(mor_wrapped_point_lon,mor_wrapped_point_lat, color='k',
                        linewidth=2, alpha=1, zorder=2, transform=data_crs)

ax.set_global()
norm = mpl.colors.Normalize(vmin=0,vmax=180)
cmap = plt.get_cmap('viridis',180)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
plt.colorbar(sm, boundaries=np.arange(0,200,20))

plt.savefig('antarctica.pdf')

In [ ]:
coastline_geom.to_lat_lon_array()[::,0]

In [ ]:
data_crs = ccrs.Geodetic()
poly_data_crs = ccrs.PlateCarree()
date_line_wrapper = pygplates.DateLineWrapper(0) 

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(24,12),subplot_kw={'projection': ccrs.Robinson(180)})
ax.scatter(seafloor_production[10][MeanLon] , seafloor_production[10][MeanLat],transform=poly_data_crs)

for coastline in reconstructed_coastlines:
    coastline_geom = pygplates.PolygonOnSphere(coastline.get_reconstructed_geometry())
    wrapped_coastlines = date_line_wrapper.wrap(coastline_geom, 1)
    for wrapped_coastline in wrapped_coastlines:
        wrapped_coastlines_lat = []
        wrapped_coastlines_lon = []
        for wrapped_coastline_point in wrapped_coastline.get_exterior_points():
            wrapped_coastline_lat_lon = wrapped_coastline_point.get_latitude(), wrapped_coastline_point.get_longitude()
            wrapped_coastlines_lat.append(wrapped_coastline_point.get_latitude())
            wrapped_coastlines_lon.append(wrapped_coastline_point.get_longitude())
        #ax.plot(wrapped_coastlines_lon, wrapped_coastlines_lat, color='k', lw=0.1, transform=data_crs)
        ax.fill(wrapped_coastlines_lon, wrapped_coastlines_lat, color='k', alpha=0.5, lw=1, transform=poly_data_crs)
#plt.xlim

In [ ]:
seafloor_production.